<a href="https://colab.research.google.com/github/robert-myers/you-had-me-at-hello/blob/master/notebooks/memorable_movie_quotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install catboost

     |████████████████████████████████| 64.0MB 46kB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import nltk
import re

from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.metrics import edit_distance
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import SGDClassifier, LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import confusion_matrix, silhouette_score, silhouette_samples
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans, MeanShift, MiniBatchKMeans
from sklearn.preprocessing import FunctionTransformer
from catboost import CatBoostClassifier

from scipy.spatial import distance

from google.colab import files

plt.style.use("seaborn")

In [4]:
nltk.download(['stopwords', 'wordnet', 'punkt', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
colab = True
if colab:
  df = pd.read_csv("https://raw.githubusercontent.com/robert-myers/you-had-me-at-hello/master/data/memorable_nonmemorable_movie_quotes_df.csv", index_col="line_id")
else:
  df = pd.read_csv("../data/memorable_nonmemorable_movie_quotes_df.csv")

In [6]:
 df[df["movie_title"].str.contains("matrix")]

,movie_title,line,levenshtein_distance_from_script,memorable?
line_id,,,,
516011,the matrix,"Tell me, Mr. Anderson, what good is a phone ca...",9,1
516533,the matrix,"But I believe that, as a species, human beings...",0,0
516175,the matrix,Welcome to the desert of the real.,13,1
516174,the matrix,This is Chicago as it exists today.,0,0
516583,the matrix,Find them and destroy them,1,1
516570,the matrix,"Can you hear me, Morpheus?",0,0
516001,the matrix,"I'm going to enjoy watching you die, Mr. Ander...",60,1
516573,the matrix,"I must get out of here, I must get free.",0,0
516505,the matrix,Never send a human to do a machine's job.,0,1


In [0]:
tokenizer = PunktSentenceTokenizer(" ".join(df["line"]))

stop_words = set(stopwords.words("english"))

def rm_stop_and_tokenize(x, join=True, remove_stop=False):
  x = re.sub('\W+', ' ', x)
  wnl = WordNetLemmatizer()
  if remove_stop:
    string = " ".join([wnl.lemmatize(word).lower() for word in x.split() if word.lower() not in stop_words])
  else:
    string = " ".join([wnl.lemmatize(word).lower() for word in x.split()])
  tokens = word_tokenize(string)
  if join:
    return " ".join(tokens)
  return tokens

df["tokenized"] = df["line"].apply(lambda x: rm_stop_and_tokenize(x, join=True, remove_stop=False))

In [0]:
df["parts_of_speech"] = df["line"].apply(lambda x: [tup[-1] for tup in nltk.pos_tag(x.split())])

In [9]:
df[df["movie_title"].str.contains("matrix")]

,movie_title,line,levenshtein_distance_from_script,memorable?,tokenized,parts_of_speech
line_id,,,,,,
516011,the matrix,"Tell me, Mr. Anderson, what good is a phone ca...",9,1,tell me mr anderson what good is a phone call ...,"[NNP, VBZ, NNP, NNP, WP, NN, VBZ, DT, NN, NN, ..."
516533,the matrix,"But I believe that, as a species, human beings...",0,0,but i believe that a a specie human being defi...,"[CC, PRP, VBP, RB, IN, DT, JJ, JJ, NNS, VBP, P..."
516175,the matrix,Welcome to the desert of the real.,13,1,welcome to the desert of the real,"[VB, TO, DT, NN, IN, DT, NN]"
516174,the matrix,This is Chicago as it exists today.,0,0,this is chicago a it exists today,"[DT, VBZ, NNP, IN, PRP, VBZ, NN]"
516583,the matrix,Find them and destroy them,1,1,find them and destroy them,"[VB, PRP, CC, VB, PRP]"
516570,the matrix,"Can you hear me, Morpheus?",0,0,can you hear me morpheus,"[MD, PRP, VB, NNS, NNP]"
516001,the matrix,"I'm going to enjoy watching you die, Mr. Ander...",60,1,i m going to enjoy watching you die mr anderson,"[NNP, VBG, TO, VB, VBG, PRP, VBP, NNP, NNP]"
516573,the matrix,"I must get out of here, I must get free.",0,0,i must get out of here i must get free,"[PRP, MD, VB, IN, IN, NN, PRP, MD, VB, NN]"
516505,the matrix,Never send a human to do a machine's job.,0,1,never send a human to do a machine s job,"[RB, VB, DT, NN, TO, VB, DT, NN, NN]"


In [10]:
df.loc[364691]["line"]

'All we want is to be treated like human beings, not like guinea pigs to be experimented on and not like bunny rabbits to be patronized.'

In [11]:
memorable_tokens = df[df["memorable?"] == 1]["tokenized"]
memorable_counter = Counter((" ".join(memorable_tokens)).split())
non_memorable_tokens = df[df["memorable?"] == 0]["tokenized"]
non_memorable_counter = Counter((" ".join(non_memorable_tokens)).split())

print(" memorable", " |",  "non-memorable")
print("----------------------------")
for x,y in zip(memorable_counter.most_common(25), non_memorable_counter.most_common(25)):
  print(x, "|", y)

 memorable  | non-memorable
----------------------------
('you', 958) | ('i', 933)
('i', 864) | ('you', 850)
('the', 745) | ('the', 728)
('a', 728) | ('to', 641)
('to', 470) | ('a', 595)
('t', 345) | ('s', 401)
('s', 328) | ('and', 389)
('of', 327) | ('it', 379)
('and', 294) | ('of', 342)
('it', 279) | ('t', 323)
('is', 266) | ('that', 290)
('that', 260) | ('in', 284)
('in', 250) | ('we', 227)
('me', 212) | ('me', 213)
('my', 208) | ('for', 177)
('your', 207) | ('what', 170)
('this', 195) | ('is', 168)
('for', 168) | ('he', 166)
('we', 166) | ('have', 165)
('don', 161) | ('your', 157)
('m', 149) | ('this', 155)
('not', 147) | ('be', 154)
('have', 144) | ('m', 154)
('be', 142) | ('my', 148)
('on', 140) | ('re', 143)


In [12]:
df["line"].loc[516011]

'Tell me, Mr. Anderson, what good is a phone call when you are unable to speak?'

In [13]:
typo = "And tell me, Mr. Anderson, what good is a phone call iy you are unable to speak?"
no_typo = "And tell me, Mr. Anderson, what good is a phone call if you are unable to speak?"
edit_distance(df["line"].loc[516011], typo) == edit_distance(df["line"].loc[516011], no_typo)

True

In [0]:
X = df.drop(columns="memorable?")
y = df["memorable?"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
def roc_curve_plot(y_test, y_probs, save_fig=False):
    '''
    This function prints and plots the ROC plot.
    '''
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_probs)
    roc_auc = metrics.auc(fpr, tpr)
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})', linewidth=2)
    plt.plot([0,1], [0,1], linestyle='--', lw=2, color='r')
    plt.xlabel("False Positive Rate (1 - Specificity)", fontsize=14)
    plt.ylabel("True Positive Rate (Sensitivity, Recall)", fontsize=14)
    plt.title("ROC plot", fontsize=20)
    plt.legend()
    if save_fig:
        plt.savefig('roc_plot.png')
    plt.show();

MultinomialNB

In [16]:
text_clf = Pipeline([
                     ("vect", CountVectorizer(ngram_range=(1,3))),
                     ("tfidf", TfidfTransformer()),
                     ("clf", MultinomialNB(fit_prior=False)),
                     ])
text_clf = text_clf.fit(X_train["tokenized"], y_train)
print(text_clf.score(X_test["tokenized"], y_test))

predicted = text_clf.predict(X_test["tokenized"])
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
totalpn = tn + fp + fn + tp
cf_mx = np.array([[tp/totalpn, fp/totalpn],[fn/totalpn, tn/totalpn]])
cf_mx

0.5868971792538672


array([[0.25204732, 0.1656051 ],
       [0.24749773, 0.33484986]])

In [17]:
np.mean(cross_val_score(text_clf, X["tokenized"], y, n_jobs=-1, cv=10))

0.6094677987160904

In [0]:
# roc_curve_plot(y_test, predicted)

In [19]:
test_string = [rm_stop_and_tokenize("You had me at hello.")]
print(test_string)
text_clf.predict(test_string)

['you had me at hello']


array([0])

GridSearchCV MultinomialNB

In [20]:
parameters = {
    # "vect__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "vect__max_features": [1000, 2000, 3000, None],
    # "tfidf__use_idf": (True, False),
    "clf__alpha": (1e-0, 1e-1, 1e-2, 1e-3),
    }
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train["tokenized"], y_train)
gs_clf.best_params_

{'clf__alpha': 1.0, 'vect__max_features': None}

In [21]:
gs_clf.best_score_

0.6018209408194235

In [22]:
predicted = gs_clf.predict(X_test["tokenized"])
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
totalpn = tn + fp + fn + tp
cf_mx = np.array([[tp/totalpn, fp/totalpn],[fn/totalpn, tn/totalpn]])
cf_mx

array([[0.25204732, 0.1656051 ],
       [0.24749773, 0.33484986]])

In [23]:
%%time
# np.mean(cross_val_score(gs_clf, X["tokenized"], y, n_jobs=-2, cv=10))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [0]:
# roc_curve_plot(y_test, predicted)

In [25]:
test_string = [rm_stop_and_tokenize("You had me at hello.")]
print(test_string)
gs_clf.predict(test_string)

['you had me at hello']


array([0])

SGDClassifier

In [26]:
text_clf_sgd = Pipeline([
                         ("vect", CountVectorizer(ngram_range=(1,3))),
                         ("tfidf", TfidfTransformer()),
                         ("clf-sgd", SGDClassifier(n_jobs=-1)),
                         ])
text_clf_sgd.fit(X_train["tokenized"], y_train)
print(text_clf_sgd.score(X_test["tokenized"], y_test))

predicted = text_clf_sgd.predict(X_test["tokenized"])
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
totalpn = tn + fp + fn + tp
cf_mx = np.array([[tp/totalpn, fp/totalpn],[fn/totalpn, tn/totalpn]])
cf_mx

0.5941765241128298


array([[0.26933576, 0.17561419],
       [0.23020928, 0.32484076]])

In [27]:
np.mean(cross_val_score(text_clf_sgd, X["tokenized"], y, n_jobs=-1, cv=10))

0.6056051977635121

In [0]:
# roc_curve_plot(y_test, predicted)

In [29]:
test_string = [rm_stop_and_tokenize("You had me at hello.")]
print(test_string)
text_clf_sgd.predict(test_string)

['you had me at hello']


array([0])

GridSearchCV SGDClassifier

In [30]:
parameters = {
    # "clf-sgd__loss": ("hinge", "log", "modified_huber"),
    # "clf-sgd__penalty": ("l2", "l1", "elasticnet"),
    "clf-sgd__alpha": (1e-2, 1e-3, 1e-4, 1e-5),
    }
gs_clf_sgd = GridSearchCV(text_clf_sgd, parameters, n_jobs=-1)
gs_clf_sgd = gs_clf_sgd.fit(X_train["tokenized"], y_train)
gs_clf_sgd.best_params_

{'clf-sgd__alpha': 0.001}

In [31]:
gs_clf_sgd.best_score_

0.6063732928679817

In [32]:
np.mean(cross_val_score(gs_clf_sgd, X["tokenized"], y, n_jobs=-2, cv=10))

0.6153903499689377

In [33]:
predicted = gs_clf_sgd.predict(X_test["tokenized"])
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
totalpn = tn + fp + fn + tp
cf_mx = np.array([[tp/totalpn, fp/totalpn],[fn/totalpn, tn/totalpn]])
cf_mx

array([[0.31847134, 0.20473157],
       [0.1810737 , 0.29572338]])

In [0]:
# roc_curve_plot(y_test, predicted)

In [35]:
test_string = [rm_stop_and_tokenize('You had me at "Hello."')]
print(test_string)
gs_clf_sgd.predict(test_string)

['you had me at hello']


array([0])

BernoulliNB

In [36]:
text_clf_bern = Pipeline([
                     ("vect", CountVectorizer(ngram_range=(1,3))),
                     ("tfidf", TfidfTransformer()),
                     ("clf", BernoulliNB()),
                     ])
text_clf_bern = text_clf_bern.fit(X_train["tokenized"], y_train)
print(text_clf_bern.score(X_test["tokenized"], y_test))

predicted = text_clf_bern.predict(X_test["tokenized"])
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
totalpn = tn + fp + fn + tp
cf_mx = np.array([[tp/totalpn, fp/totalpn],[fn/totalpn, tn/totalpn]])
cf_mx

0.5878070973612375


array([[0.28025478, 0.19290264],
       [0.21929026, 0.30755232]])

In [37]:
np.mean(cross_val_score(text_clf_bern, X["tokenized"], y, n_jobs=-1, cv=10))

0.6044626216607993

In [0]:
# roc_curve_plot(y_test, predicted)

In [39]:
test_string = [rm_stop_and_tokenize("You had me at hello.")]
print(test_string)
text_clf_bern.predict(test_string)

['you had me at hello']


array([0])

RandomForestClassifier

In [40]:
text_clf_rfc = Pipeline([
                    ("vect", CountVectorizer(ngram_range=(1,3))),
                    ("tfidf", TfidfTransformer()),
                    ("clf", RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=-1)),
                    ])
text_clf_rfc = text_clf_rfc.fit(X_train["tokenized"], y_train)
print(text_clf_rfc.score(X_test["tokenized"], y_test))

predicted = text_clf_rfc.predict(X_test["tokenized"])
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
totalpn = tn + fp + fn + tp
cf_mx = np.array([[tp/totalpn, fp/totalpn],[fn/totalpn, tn/totalpn]])
cf_mx

0.5814376706096451


array([[0.19017288, 0.10919017],
       [0.30937216, 0.39126479]])

In [41]:
np.mean(cross_val_score(text_clf_rfc, X["tokenized"], y, n_jobs=-1, cv=10))

0.5869408780285774

In [0]:
# roc_curve_plot(y_test, predicted)

In [43]:
test_string = [rm_stop_and_tokenize("You had me at hello.")]
print(test_string)
text_clf_rfc.predict(test_string)

['you had me at hello']


array([0])

CatBoostClassifier

In [44]:
text_clf_cat = Pipeline([
                    ("vect", CountVectorizer(ngram_range=(1,3))),
                    ("tfidf", TfidfTransformer()),
                    ("clf", CatBoostClassifier(verbose=False)),
                    ])
text_clf_cat = text_clf_cat.fit(X_train["tokenized"], y_train)
predicted = text_clf_cat.predict(X_test["tokenized"])

np.mean(y_test == predicted)

0.5677888989990901

In [45]:
%%time
# np.mean(cross_val_score(text_clf_cat, X["tokenized"], y, n_jobs=-2, cv=10))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [0]:
# roc_curve_plot(y_test, predicted)

In [47]:
test_string = [rm_stop_and_tokenize("You had me at hello.")]
print(test_string)
text_clf_cat.predict(test_string)

['you had me at hello']


array([0])

LinearSVC

In [48]:
text_clf_lsvc = Pipeline([
                    ("vect", CountVectorizer(ngram_range=(1,3))),
                    ("tfidf", TfidfTransformer()),
                    ("clf", LinearSVC()),
                    ])
text_clf_lsvc = text_clf_lsvc.fit(X_train["tokenized"], y_train)
print(text_clf_lsvc.score(X_test["tokenized"], y_test))

predicted = text_clf_lsvc.predict(X_test["tokenized"])
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
totalpn = tn + fp + fn + tp
cf_mx = np.array([[tp/totalpn, fp/totalpn],[fn/totalpn, tn/totalpn]])
cf_mx

0.5923566878980892


array([[0.26842584, 0.17652411],
       [0.2311192 , 0.32393085]])

In [49]:
np.mean(cross_val_score(text_clf_lsvc, X["tokenized"], y, n_jobs=-1, cv=10))

0.611518948022365

In [0]:
# roc_curve_plot(y_test, predicted)

In [51]:
test_string = [rm_stop_and_tokenize("You had me at hello.")]
print(test_string)
text_clf_lsvc.predict(test_string)

['you had me at hello']


array([0])

Clustering

In [0]:
# for n_clusters in range(2,10):
#   text_clstr = Pipeline([
#                       ("vect", CountVectorizer(ngram_range=(1,3))),
#                       ("tfidf", TfidfTransformer()),
#                       ("to_dense", FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)), 
#                       ("clstr", KMeans(n_clusters=n_clusters, n_jobs=-1)),
#                       ])
#   text_clstr = text_clstr.fit(df[df["memorable?"] == 1]["tokenized"])
#   cluster_labels = text_clstr["clstr"].fit_predict(text_clstr["tfidf"].idf_.reshape(-1, 1))
#   print(n_clusters)
#   print(silhouette_score(text_clstr["tfidf"].idf_.reshape(-1, 1), cluster_labels))

In [53]:
%%time

X_feats = Pipeline([
                    ("vect", CountVectorizer(ngram_range=(1,3))),
                    ("tfidf", TfidfTransformer()),
                    ("to_dense", FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
                    ])
X_feats = X_feats.fit_transform(df["line"])

range_n_clusters = range(2,9)
for n_clusters in range_n_clusters:
  mbkmeans = MiniBatchKMeans(n_clusters=n_clusters)
  cluster_labels = mbkmeans.fit_predict(X_feats)

  silhouette_avg = silhouette_score(X_feats, cluster_labels)
  print("For n_clusters =", n_clusters,
        "The average silhouette_score is :", silhouette_avg)


For n_clusters = 2 The average silhouette_score is : 0.0004567327567487123
For n_clusters = 3 The average silhouette_score is : -0.0003908667247347646
For n_clusters = 4 The average silhouette_score is : -0.0017756396417998174
For n_clusters = 5 The average silhouette_score is : -0.0013142856267943643
For n_clusters = 6 The average silhouette_score is : -0.0020173697522622752
For n_clusters = 7 The average silhouette_score is : -0.005479969984197092
For n_clusters = 8 The average silhouette_score is : -0.004392534430039383
CPU times: user 8min 19s, sys: 55.7 s, total: 9min 15s
Wall time: 4min 45s


In [0]:
mini_km = MiniBatchKMeans(n_clusters=2).fit(X_feats)

In [0]:
euc_arr0 = []
euc_arr1 = []
for row in X_feats:
  euc_arr0.append(distance.euclidean(row, mini_km.cluster_centers_[0]))
  euc_arr1.append(distance.euclidean(row, mini_km.cluster_centers_[1]))
euc_arr0 = np.array(euc_arr0)
euc_arr1 = np.array(euc_arr1)

In [0]:
# cos_arr0 = []
# cos_arr1 = []
# for row in array_text:
#   cos_arr0.append(distance.cosine(row, mini_km.cluster_centers_[0]))
#   cos_arr1.append(distance.cosine(row, mini_km.cluster_centers_[1]))
# cos_arr0 = np.array(cos_arr0)
# cos_arr1 = np.array(cos_arr1)

In [60]:
Counter(mini_km.labels_)

Counter({0: 259, 1: 4135})

In [61]:
euc_arr0.argsort()

array([2470, 1182, 3162, ..., 2022, 3444, 3136])

In [62]:
euc_arr1.argsort()

array([3681, 3079,  849, ...,  701, 2120, 2022])

In [64]:
np.sum(X_feats[2262])

2.449489742783178

In [65]:
rm_stop_and_tokenize("I despise a floggin' rooster.	")

'i despise a floggin rooster'

In [68]:
df.iloc[euc_arr0.argsort()[:10]]

,movie_title,line,levenshtein_distance_from_script,memorable?,tokenized,parts_of_speech
line_id,,,,,,
492109,lost in space,"We're lost, aren't we?",0,1,we re lost aren t we,"[NNP, VBP, NN, NN]"
854322,wag the dog,"We're not gonna have a war, we're gonna have t...",18,1,we re not gon na have a war we re gon na have ...,"[NNP, RB, RB, VBP, DT, NN, NN, NN, VBP, DT, NN..."
117279,blade runner,"We're not computers, Sebastian, we're physical.",0,1,we re not computer sebastian we re physical,"[NNP, RB, VB, NNP, NN, NN]"
796532,the third man,If we have to see the Porter we'd better go.,0,0,if we have to see the porter we d better go,"[IN, PRP, VBP, TO, VB, DT, NNP, NN, RBR, NN]"
683671,shallow grave,Now this is what we need.,0,0,now this is what we need,"[RB, DT, VBZ, WP, PRP, VBP]"
225751,the departed,We've been after this cocksucker for a long ti...,0,0,we ve been after this cocksucker for a long ti...,"[NNP, VBN, IN, DT, NN, IN, DT, JJ, NN, CC, NN,..."
574730,"o brother, where art thou?",I know we're kin!,0,0,i know we re kin,"[PRP, VBP, JJ, NN]"
535346,mission to mars,"When we get back, we have GOT to try this in t...",29,1,when we get back we have got to try this in th...,"[WRB, PRP, VBP, IN, PRP, VBP, VBN, TO, VB, DT,..."
284470,final destination 2,We're *all* going to die.,68,1,we re all going to die,"[NNP, VBZ, VBG, TO, VB]"


In [66]:
df.iloc[euc_arr1.argsort()[:10]]

,movie_title,line,levenshtein_distance_from_script,memorable?,tokenized,parts_of_speech
line_id,,,,,,
645373,revolutionary road,And what do you think you're going to do?,0,0,and what do you think you re going to do,"[CC, WP, VBP, PRP, VB, RB, VBG, TO, VB]"
683835,shallow grave,"You're not going to take it to a bank Or what,...",0,0,you re not going to take it to a bank or what ...,"[MD, RB, VBG, TO, VB, PRP, TO, DT, NN, CC, VB,..."
233874,dogma,...and now you're telling me that I'm supposed...,0,0,and now you re telling me that i m supposed to...,"[NN, RB, VBZ, VBG, PRP, IN, NNP, VBD, TO, VB, ..."
698263,the sixth sense,You don't have to tell me your secret if you d...,0,0,you don t have to tell me your secret if you d...,"[PRP, VBP, VB, TO, VB, PRP, PRP$, JJ, IN, PRP,..."
545759,mumford,I agree with you that you don't know what you'...,0,0,i agree with you that you don t know what you ...,"[PRP, VBP, IN, PRP, IN, PRP, VBP, VB, WP, VB, ..."
694505,silence of the lambs,You're so close to the way you're going to cat...,0,0,you re so close to the way you re going to cat...,"[NNP, RB, RB, TO, DT, NN, VBZ, VBG, TO, VB, PR..."
225648,the departed,Do you want him to chop me up and feed me to t...,45,1,do you want him to chop me up and feed me to t...,"[VBP, PRP, VB, PRP, TO, VB, PRP, RP, CC, VB, P..."
711742,someone to watch over me,What do you want to do?,0,0,what do you want to do,"[WP, VBP, PRP, VB, TO, VB]"
411547,interview with the vampire: the vampire chroni...,So you want me to tell you the story of my life?,3,1,so you want me to tell you the story of my life,"[RB, PRP, VBP, PRP, TO, VB, PRP, DT, NN, IN, P..."


In [0]:
# cluster_labels = text_clstr["clstr"].fit_predict(text_clstr["tfidf"].idf_.reshape(-1, 1))

# silhouette_score(text_clstr["tfidf"].idf_.reshape(-1, 1), cluster_labels)

In [0]:
text = Pipeline([
                 ("vect", CountVectorizer(ngram_range=(1,3))),
                 ("tfidf", TfidfTransformer()),
                 ])
# text_clstr = text_clstr.fit((df[df["memorable?"] == 1]["tokenized"]))

text = text.fit_transform(df["tokenized"])
df["tfidf"] = list(text)
# df["tfidf"] = df["tfidf"].apply(lambda x: x.toarray())

In [70]:
df.iloc[0]["tfidf"].toarray().shape

(1, 57330)

In [0]:
#

In [74]:
vocab = set((" ".join(df["tokenized"]).split()))
len(vocab)

5636

In [75]:
class_counts = Counter(y)
class_counts

Counter({0: 2197, 1: 2197})

In [76]:
total_docs = sum(class_counts.values())
total_docs

4394

In [77]:
words_per_class = defaultdict(int)
word_count_per_class = defaultdict(Counter)

for doc, label in zip(X["tokenized"],y):
    words_per_class[label] += len(doc.split())
    word_count_per_class[label].update(Counter(doc.split()))

words_per_class

defaultdict(int, {0: 22569, 1: 22565})